# Description

This notebook will look at a rudimentary way of generating images using stock images of classes observed by the Raspberry Pi and an associated json annotation for the object perceived and it's location. The PIL library has a lot of useful functions which can refine this notebook but this is just a quick look.

**Imports**

In [2]:
from PIL import Image
from random import randint
import pandas as pd
import requests
from io import BytesIO
import zipfile
import numpy as np
import yapf
from io import BytesIO

# Loading dataset

In [3]:
# Generated using gaussian copula trained on original telemetric dataset
dftele = pd.read_json('./data/generated/telemetric_for_image.json')
dftele.columns = ['types', 'classes', 'scores', 'x1', 'y1', 'x2', 'y2']
dftele.head()

,types,classes,scores,x1,y1,x2,y2
0,c,person,0.522133,0.575799,0.700929,0.622501,0.908469
1,c,couch,0.571273,0.455880,0.404786,0.877082,0.783824
2,c,couch,0.537385,0.503401,0.041081,0.712458,0.370436
3,c,chair,0.611639,0.478243,0.563103,0.640052,0.664028
4,c,person,0.521420,0.327983,0.241282,0.633186,0.244069


In [4]:
dftele = dftele[['classes', 'x1', 'y1', 'x2', 'y2']] # Selecting important features
dftele.head()

,classes,x1,y1,x2,y2
0,person,0.575799,0.700929,0.622501,0.908469
1,couch,0.455880,0.404786,0.877082,0.783824
2,couch,0.503401,0.041081,0.712458,0.370436
3,chair,0.478243,0.563103,0.640052,0.664028
4,person,0.327983,0.241282,0.633186,0.244069


In [5]:
dftele.classes.value_counts()

chair     321
couch     291
person    195
tv        155
laptop     29
tie         9
Name: classes, dtype: int64

# Loading images
These are stock images with one image for each class but if required each class could have a list of images all which could describe the colour and shape of the object too to generate randomly assorted images

In [82]:
couch = Image.open('./data/objects/couch.png').convert('RGBA')
tv = Image.open('./data/objects/tv.png').convert('RGBA')
person = Image.open('./data/objects/person.jfif').convert('RGBA')
chair = Image.open('./data/objects/chair.png').convert('RGBA')
car = Image.open('./data/objects/car.png').convert('RGBA')
laptop = Image.open('./data/objects/laptop.png').convert('RGBA')
tie = Image.open('./data/objects/tie.png').convert('RGBA')

images = [couch, tv, person, chair, car, laptop, tie]

In [83]:
background = Image.open('./data/objects/white_background.jfif').convert('RGBA')
background = background.resize((256, 256), resample=PIL.Image.LANCZOS)

# Generating images 

In [50]:
# This can be used to keep images in their original aspect. However, given we have boxes indicating where the object should 
# be I decided to fill the entire box with each object
aspects = [image.width / image.height for image in images]

In [51]:
dispatcher = {
    'couch': couch,
    'tv': tv,
    'person': person,
    'car': car,
    'laptop': laptop,
    'tv': tv,
    'chair': chair,
    'tie' : tie
} # Allows objects to be used from strings

In [78]:
def renderer(dataframe, back, zip_name):
    # Initializing the file
    OFFSET = 1
    z = zipfile.ZipFile(f'./data/generated/{zip_name}.zip', 'w',
                        zipfile.ZIP_DEFLATED)
    
    for i in range(dataframe.classes.size):
        render_img = back.copy() # Ensuring background does not get clutured
        
        # to keep aspect the same for the pictures changes these lines
        w = int(abs(dataframe.loc[i].x2 - abs(dataframe.loc[i].x1)) * 256)
        if w == 0: w += 1 # to ensure non 0 values
        h = int(abs(dataframe.loc[i].y2 - abs(dataframe.loc[i].y1)) * 256)
        if h == 0: h += 1

        item = dispatcher[dataframe.loc[i].classes].resize(
            (w, h), resample=PIL.Image.LANCZOS) # Rezises images

        # Finds x and y coordinate of boxes
        x = int(((dataframe.loc[i].x1 + dataframe.loc[i].x2) / 2) * 256) 
        y = int(((dataframe.loc[i].y1 + dataframe.loc[i].y2) / 2) * 256)

        render_img.paste(item, (x, y), item) # pastes image onto background

        # Saves new image
        image_file = BytesIO()
        render_img.save(image_file, 'PNG')
        z.writestr(f'objects-{i+OFFSET}.png', image_file.getvalue())
    z.writestr('master.json', dataframe.to_json())
    z.close()
    print("done")
    return None

In [80]:
a = renderer(dftele, background, 'Complete')

done
